Chapter 10. Advanced Expressions
===================

In [ ]:
from proveit import ExprTuple
from proveit._common_ import x, y, z
%begin advanced_expressions
xyz = ExprTuple(x, y, z)

In [ ]:
xy = ExprTuple(x, y)

The `get_elem` method may be used to access an element of the `ExprTuple` using an `Expression` representative for the index, relative to a `base` (defaulting to `1`, common in mathematics though it is less common in programming).

In [ ]:
from proveit.numbers import num
xyz.get_elem(num(3)) # get the 3rd element of (x, y, z)

The method works by starting from the `base` and adding `1` until the given index is reached while iterating through the elements of the `ExprTuple`.  It uses the `proveit.numbers` package for number representations for generating the required counting steps which are optionally passed back via the `requirements` parameter.

In [ ]:
requirements = []
xyz.get_elem(num(2), base=0, requirements=requirements) # start from a base of zero and pass back the requirements

In [ ]:
ExprTuple(*requirements) # composing an ExprTuple out of the requirements list is done to display a LaTeX representation.

The *index* argument into the `get_elem` method does not need to be a direct representation of a number.  If it can be proven to be equal to a counted index value (through *assumptions*, for example), that also works.

In [ ]:
from proveit._common_ import k
from proveit.logic import Equals
requirements = []
xyz.get_elem(k, assumptions=[Equals(k, num(2))], requirements=requirements)

In [ ]:
ExprTuple(*requirements)

<a name="Indexed"></a>IndexedVar (to be updated)
====================

Before we discuss `Iter`s, let us first introduced `Indexed` which are designed specifically to be contained in an `Iter`.  An `Indexed` `Expression` has a *variable* (as a `Variable`) and an *index* (or *indices*).  The *variable* is intended to be a placeholder for an `ExprTuple` (or `ExprArray`).  The `Indexed` object represents an element of that `ExprTuple` (or `ExprArray`) addressed by the *index* (or *indices*).  Here is a simple example for a single *index* $k$ of *variable* $a$:

In [ ]:
from proveit import IndexedVar
from proveit._common_ import a, b, k
a_k = IndexedVar(a, k)

In [ ]:
a_k.expr_info()

Note that a *base* is shown in the *expression information*.  This indicates an initial offset into the list.  The default is *base*=1 which means that 1 is taken to be the first element.  Although this *base* is not shown in the LaTeX rendering (or the string rendering), it can usually be inferred from the theory of a containing `Iter`.  Let us show a *base*=0 example:

In [ ]:
b_k = IndexedVar(b, k, base=0)

In [ ]:
b_k.expr_info()

The *index* may be substituted freely.

In [ ]:
from proveit.logic import And
a_and_b = And(a, b)
a_k.substituted({k:a_and_b})

The *variable* may only be substituted with another `Variable` or with an `ExprTuple`.

In [ ]:
a_k.substituted({a:b})

In [ ]:
try:
    a_k.substituted({a:a_and_b})
    assert False, "Expecting an TypeError error; should not make it to this point"
except TypeError as e:
    print("EXPECTED ERROR:", e)

But when the *variable* is substituted with an `ExprTuple`, the *index* must be an `Expression` that is "known" to evaluate to a specific element (or iteration range as we see below in the `Iter` section).

In [ ]:
from proveit import ExprTupleError
try:
    a_k.substituted({a:xyz})
    assert False, "Expecting an ExprTupleError error; should not make it to this point"
except ExprTupleError as e:
    print("EXPECTED ERROR:", e)

Here we assign the index, $k$, to a specific numeric value to extract the first element of the $(x, y, z)$:

In [ ]:
from proveit.numbers import num
a_k.substituted({a:xyz, k:num(1)})

But we can also assign the index implicitly via *assumptions* or other known relationships:

In [ ]:
from proveit.logic import Equals
from proveit.numbers import num
requirements=[]
a_k.substituted({a:xyz}, assumptions=[Equals(k, num(2))], requirements=requirements)

As with the `get_elem` of the `ExprTuple` class, we can pass back the requirements for counting into the `ExprTuple` that is being substituted into the *indexed* *variable*, as attained above and displayed below.

In [ ]:
ExprTuple(*requirements)

Next, we see what happens when the *index* is out-of-range.

In [ ]:
try:
    a_k.substituted({a:xyz, k:num(4)})
    assert False, "Expecting an IndexError error; should not make it to this point"
except IndexError as e:
    print("EXPECTED ERROR:", e)

<a name="Iter"></a>Iter
====================

An `Iter` `Expression` represents an iteration over a range.  For example,

In [ ]:
from proveit import Iter
from proveit._common_ import i, j
a_it = Iter(i, IndexedVar(a, i), num(1), num(3))

In [ ]:
ExprTuple(a_it).get_elem(num(2))

In [ ]:
%end advanced_expressions

In [ ]:
from proveit.numbers import Less, num
Less(num(3), num(4)).prove().derive_relaxed()

In [ ]:
ExprTuple(a_it).get_elem(num(4))

In [ ]:
ExprTuple(a_it, b_it).get_elem(num(2))

Here, `a_it` represents a list ranging from the first element of $x$ to the third element of $x$.  If we substitute $x$ with an `ExprTuple`, the iteration will be expanded by successively applying `+1` (via `proveit.numbers` definitions of addition and `1`) to the *start index* (1 in this case) until the *end index* (3 in this case) is reached.

In [ ]:
a_it.substituted({a:xyz})

In the following case, the substituted ExprTuple is not long enough to cover the iteration range.

In [ ]:
try:
    a_it.substituted({a:xy})
    assert False, "Expecting an IndexError error; should not make it to this point"
except IndexError as e:
    print("EXPECTED ERROR:", e)

The `Iter` may be embedded within a containing `ExprTuple`.

In [ ]:
b_it = Iter(j, IndexedVar(b, j), num(1), k)
a13_b1k_yz = ExprTuple(a_it, b_it, y, z)

When expanded via *expression substition*, the result is inserted into the outer `ExprTuple`:

In [ ]:
a13_b1k_yz.substituted({a:xyz, b:xy, k:num(2)})

In [ ]:
a13_b1k_yz.substituted({a:xyz, b:ExprTuple(x, a_it, y), k:num(5)})

In [ ]:
a13_b1k_yz.substituted({a:xyz, b:ExprTuple(a_it), k:num(3)})

A simple `Iter` over an `IndexedVar` as above may be used as `Lambda` *parameters* or `Operation` *operators*.  For example,

In [ ]:
from proveit.numbers import Add
Lambda(a13_b1k_yz, Add(*a13_b1k_yz))

Note that the asterisk, `*`, above is necessary to indicate addition over the `ExprTuple` rather than `Add` applied to a single `ExprTuple` term.

An `Iter` may be applying more generally than an iteration over an **indexed** **variable**.

In [ ]:
Iter(Lambda(k, Add(k, k)), num(1), num(5))

<a name="ExprArray"></a>ExprArray
====================

An `ExprArray` is the multi-dimensional analog to `ExprTuple`.  It has not been fully implemented, however.  This tutorial will be updated after it has been implemented.

# Next chapter: <a href="tutorial11_advanced_proofs.ipynb">Proofs Using Advanced Expressions</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>